# HW 2
## Named Entety Recognition and Event Extraction from Literary Fiction

deadline: 12 декабря 2022, 23:59

В этом домашнем задании вы будете работать с корпусом LitBank. Корпус собран из популярных художественных произведений на английском языке и сожержит разметку по именованным сущностям и событиям. Объем корпуса таков: 100 текстов по примерно 2000 слов каждый. 

Корпус описан в статьях:
* David Bamman, Sejal Popat, Sheng Shen, An Annotated Dataset of Literary Entities http://people.ischool.berkeley.edu/~dbamman/pubs/pdf/naacl2019_literary_entities.pdf
* Matthew Sims, Jong Ho Park, David Bamman, Literary Event Detection,  http://people.ischool.berkeley.edu/~dbamman/pubs/pdf/acl2019_literary_events.pdf

Корпус доступен в репозитории проекта:  https://github.com/dbamman/litbank

Статья и код, использованный для извлечения именованных сущностей: 
* Meizhi Ju, Makoto Miwa and Sophia Ananiadou, A Neural Layered Model for Nested Named Entity Recognition, https://github.com/meizhiju/layered-bilstm-crf

Структура корпуса устроена так. 
Первый уровень: 
* entities -- разметка по сущностям
* events -- разметка по сущностям


В корпусе используются 6 типов именованных сущностей: PER, LOC, ORG, FAC, GPE, VEH (имена, локации, организации, помещения, топонимы, средства перемещния), допускаются вложенные сущности. 

События выражается одним словом - *триггером*, которое может быть глагом, прилагательным и существительным. В корпусе описаны события, которые действительно происходят и не имеют гипотетического характера. 
Пример: she *walked* rapidly and resolutely, здесь *walked* -- триггер события. Типы событий не заданы. 



Второй уровень:
* brat -- рабочие файлы инструмента разметки brat, ann-файлы содержат разметку, txt-файлы – сырые тексты 
* tsv -- tsv-файлы содержат разметку в IOB формате,


В статье и репозитории вы найдете идеи, которые помогут вам выполнить домашнее задание. Их стоит воспринимать как руководство к действию, и не стоит их копировать и переиспользовать. Обученные модели использовать не нужно, код для их обучения можно использовать как подсказку. 

## ПРАВИЛА
1. Домашнее задание выполняется в группе до 4-х человек.
2. Домашнее задание сдается через github.classroom, инвайты будут высланы.
3. Домашнее задание оформляется в виде отчета либо в .pdf файле, либо ipython-тетрадке. 
4. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
5. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
6. Отчеты, состоящие исключительно из кода, не будут проверены и будут автоматически оценены нулевой оценкой.
7. Плагиат и любое недобросоветсное цитирование приводит к обнуление оценки. 


## Base imports

In [6]:
from typing import *
import re
from pathlib import Path
import shutil
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import joblib
%load_ext autoreload
%autoreload 2

## Dataset download and init

In [3]:
%cd data
!git clone https://github.com/dbamman/litbank
%cd ..

Cloning into 'litbank'...
remote: Enumerating objects: 1187, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 1187 (delta 26), reused 22 (delta 22), pack-reused 1149
Receiving objects: 100% (1187/1187), 40.65 MiB | 4.30 MiB/s, done.
Resolving deltas: 100% (152/152), done.
Updating files: 100% (1423/1423), done.


In [77]:
from utils import LitbankDataset

data_dir = Path('data/litbank')
dataset = LitbankDataset(data_dir)

# Словари с датафреймом разметки для каждого текста
entity_df_dict = dataset.df_dicts['entities']
event_df_dict = dataset.df_dicts['events']
# Объединенные датафреймы
entity_df_all = dataset.dfs['entities']
event_df_all = dataset.dfs['events']

In [80]:
entity_df_dict.keys()

dict_keys(['541_the_age_of_innocence_brat.tsv', '2891_howards_end_brat.tsv', '2775_the_good_soldier_brat.tsv', '6593_history_of_tom_jones_a_foundling_brat.tsv', '44_the_song_of_the_lark_brat.tsv', '32_herland_brat.tsv', '271_black_beauty_brat.tsv', '4300_ulysses_brat.tsv', '351_of_human_bondage_brat.tsv', '940_the_last_of_the_mohicans_a_narrative_of_1757_brat.tsv', '238_dear_enemy_brat.tsv', '4217_a_portrait_of_the_artist_as_a_young_man_brat.tsv', '2084_the_way_of_all_flesh_brat.tsv', '233_sister_carrie_a_novel_brat.tsv', '974_the_secret_agent_a_simple_tale_brat.tsv', '1695_the_man_who_was_thursday_a_nightmare_brat.tsv', '5348_ragged_dick_or_street_life_in_new_york_with_the_bootblacks_brat.tsv', '514_little_women_brat.tsv', '730_oliver_twist_brat.tsv', '78_tarzan_of_the_apes_brat.tsv', '160_the_awakening_and_selected_short_stories_brat.tsv', '345_dracula_brat.tsv', '45_anne_of_green_gables_brat.tsv', '24_o_pioneers_brat.tsv', '2641_a_room_with_a_view_brat.tsv', '36_the_war_of_the_world

In [74]:
entity_df_all

,token,label0,label1,label2,label3,label4
0,Book,O,O,O,O,0
1,I,O,O,O,O,0
2,I.,O,O,O,O,0
3,On,O,O,O,O,0
4,a,O,O,O,O,0
...,...,...,...,...,...,...
2058,shocking,O,O,O,O,0
2059,walk,O,O,O,O,0
2060,.,O,O,O,O,0
2061,”,O,O,O,O,0


In [76]:
event_df_all

,token,label
0,Book,O
1,I,O
2,I.,O
3,On,O
4,a,O
...,...,...
2058,shocking,O
2059,walk,EVENT
2060,.,O
2061,”,O



## Часть 1. [3 балла] Эксплоративный анализ
1. Найдите топ 10 (по частоте) именованных сущностей каждого из 6 типов.
2. Найдите топ 10 (по частоте) частотных триггеров событий.
3. Кластеризуйте все уникальные триггеры событий, используя эмбеддинги слов и любой алгоритм кластеризации (например, агломеративный иерархический алгоритм кластеризации) и попробуйте проинтерпретировать кластеры: есть ли очевидные типы событий?

[бонус] Визуализируйте полученные кластеры с помощью TSNE или UMAP

[бонус] Постройте тематическую модель по корпусу и сравните кластеры тригеров и выделенные темы: есть ли схожие паттерны в тематической модели и в стурктуре кластеров?

В следующих частях домашнего задания вам понадобится train-test-dev разбиение. Авторы статей предлагают следующую структуру разбиения: обучающее множество – 80 книг, валидационное – 10 книг, тестовое – 10 книг. Предложения из одного источника не должны попадать в разные сегменты разбиения.


## Часть 2. [5 баллов] Извлечение именованных сущностей
1. Используйте стандартную (любую предобученную) модель для извлечения именованных сущностей. Продемонстрируйте, какие сущности она извлекает. Вычислите качество работы модели на токенах и на спанах сущностей. Для вычисления качества работы модели используйте seqeval (умеет работать с Huggingface). Какая из метрик получилась выше?

2. Дообучите BERT для извлечения именованных сущностей.

2. (Как вариант альтернативный обучению BERT) Можно обучить модель CNN-BiLSTM-CRF, для извлечения именованных *низкоуровневых именованных сущностей*, т.е. для самых коротких из вложенных сущностей.
Модель устроена так: сверточная сеть на символах + эмбеддинги слов + двунаправленная LSTM сеть (модель последовательности) + CRF (глобальная нормализация)

[бонус] Используйте модель для извлечения вложенных именованных сущностей [Ju et al., 2018] Можно использовать модель из статьи, можно также вместо эмбеддингов слов использовать ELMo и/или BERT.


## Часть 3. [2 балла] Извлечение событий

1. Используйте BiLSTM на эмбеддингах слов для извлечения триггеров событий.

2. Замените часть модели на  словах  на ELMo и/или BERT.  Должна получиться модель ELMo / BERT + BiLSTM.

[бонус] Предобучите BiLSTM как языковую модель. Дообучите ее для извлечения триггеров.

[бонус] Дообучите BERT для извлечения триггеров событий.


## Часть 4. [2 балла] Одновременное извлечение именованных сущностей и событий
1. Обучите модель для совместного извлечения именованных сущностей и триггеров событий. У модели должен быть общий энкодер (например, BERT, CNN + BiLSMT, ELMo + BiLSTM, BERT + BiLSTM) и два декодера: один отвечает за извлечение именнованных сущностей, другой отвечает за извлечение триггеров событий.

[бонус] Добавьте в модель механизм внимания, так, как это покажется вам разумным.

[бонус] Визуализируйте карты механизма внимания.


## Часть 5. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Сравните результаты всех разработанных моделей. Что помогло вам в выполнении работы, чего не хватало?